In [2]:
from itertools import combinations
import pandas as pd
import numpy as np

# Data Processing

In [3]:
samples = []
with open("chainstoreFIM.txt", "r") as file:
    for x in file.readlines():
        temp = x.replace("\n", "").split(" ")
        samples.append(set(temp))

In [ ]:
len(samples)

In [5]:
len(set.union(*samples))

46086

# Naive Apriori Algorithm

Little Change from Gene Dataset. This data is massive as compared to Gene Data. So we have to limit max size of itemset before even starting the algorithm. This code is written explictly for non distirbuted environment.

In [12]:
class NaiveApriori:
    def __init__(self, data, min_support, min_confidence, max_length=None):
        self.data = data
        self.chars = set.union(*data)
        self.counter = {0: {}}
        self.rules = set()
        self.max_length = max_length
        
        # Thresholds
        self.min_support = min_support
        self.min_confidence = min_confidence
        
        # Metrics
        self.get_confidence = lambda x, y: self.get_support(set.union(x, y))/self.get_support(x)
        self.get_lift = lambda x, y: self.get_confidence(x, y)/self.get_support(y)
    
        # Utility
        self.__get_key = lambda x: "-".join(sorted(x))
        self.__get_set = lambda x: set(x.split("-"))
        
        
    def __dict_merger(self):
        temp = {}
        for x in self.counter.values():
            temp = {**temp, **x}
        self.counter = temp
        
    def __build_counter(self, ngram):
        # Build counter for item length small than the given item length, if not exist.
        if (ngram-1) not in self.counter.keys():
            self.__build_counter(ngram-1)

        # Prevent creating counter for longer itemlist is support is already exhausted for smaller counter list.
        if (ngram-1) in self.counter:
            if len(self.counter[ngram-1]) > 0 or (ngram-1) == 0:
                itemset = None
                if ngram == 1:
                    # Generate all combinations of items.
                    itemset = list(combinations(self.chars, ngram))
                else:
                    # Apriori property.
                    itemset = list(combinations(set("-".join((self.counter[ngram-1]).keys()).split("-")), ngram))

                # Count for itemlist of length ngram.
                item_count = {self.__get_key(x): 0 for x in itemset}
                for x in itemset:
                    for y in self.data:
                        x = set(x)
                        if len(set.intersection(x, y)) >= ngram :
                            item_count[self.__get_key(x)] += 1
                
                # Drop item combiantion with lower support than threshold.
                keys = list(item_count.keys())
                for key in keys:
                    if item_count[key] < self.min_support:
                        item_count.pop(key)

                self.counter[ngram] = item_count
    
    def get_support(self, key):
        key = self.__get_key(key)
        if key not in self.counter.keys():
            return 0
        return self.counter[key]
    
    def __rule_mining(self):
        keys = list(self.counter.keys())
        for i in range(len(self.counter)-1):
            for j in range(i+1, len(self.counter)):
                a = self.__get_set(keys[i])
                b = set.difference(self.__get_set(keys[j]), self.__get_set(keys[i]))
                con = self.get_confidence(a, b)
                if con >= self.min_confidence:
                    self.rules.add((self.__get_key(a), self.__get_key(b), round(con, 4)))
                con = self.get_confidence(b, a)
                if con >= self.min_confidence:
                    self.rules.add((self.__get_key(b), self.__get_key(a), round(con, 4)))

    def fit(self):
        if self.max_length is None:
            self.__build_counter(len(self.chars))
        else:
            self.__build_counter(self.max_length)
        self.__dict_merger()
        self.__rule_mining()

# Implementation

### Toy Dataset

In [7]:
s_sample = [set(list(x)) for x in ["ABE", "BD", "BC", "ABD", "AC", "BC", "AC", "ABCE", "ABC"]]
s_sample

[{'A', 'B', 'E'},
 {'B', 'D'},
 {'B', 'C'},
 {'A', 'B', 'D'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'C'},
 {'A', 'B', 'C', 'E'},
 {'A', 'B', 'C'}]

In [8]:
obj = NaiveApriori(s_sample, 2, 0.60)
obj.fit()

In [9]:
obj.counter

{'E': 2,
 'B': 7,
 'C': 6,
 'D': 2,
 'A': 6,
 'B-D': 2,
 'B-E': 2,
 'B-C': 4,
 'A-B': 4,
 'A-E': 2,
 'A-C': 4,
 'A-B-E': 2,
 'A-B-C': 2}

In [10]:
# {a, b, c} refers to rule: "a implies b with confidence 100*c %"

obj.rules

{('A', 'B', 0.6667),
 ('A', 'C', 0.6667),
 ('A-E', 'B', 1.0),
 ('B-E', 'A', 1.0),
 ('C', 'A', 0.6667),
 ('C', 'B', 0.6667),
 ('D', 'B', 1.0),
 ('E', 'A', 1.0),
 ('E', 'A-B', 1.0),
 ('E', 'B', 1.0)}

### Grocery Dataset

In [ ]:
obj = NaiveApriori(samples, 10000, 0.70, 1)
obj.fit()

In [ ]:
obj.counter.keys()

In [ ]:
obj.rules